In [1]:
import threading
import hazelcast
import time


# Функція для інкременту значення лічильника з використанням Lock для синхронізації
def with_lock(key):
    global main_counter
    thread_lock = threading.Lock()
    for _ in range(10000):
        with thread_lock:
            main_counter += 1
            distributed_map.put(key, main_counter)


# Функція для інкременту значення лічильника без Lock, з простим читанням і записом
def no_lock(key):
    distributed_map.put(key, 0)
    for _ in range(10000):
        counter = distributed_map.get(key) + 1
        distributed_map.put(key, counter)


# Функція для інкременту значення лічильника з використанням Lock-мапи (map.lock)
def pessimistic_map(key):
    if (not distributed_map.contains_key(key)): distributed_map.put(key,0)
    for _ in range(10000):
        distributed_map.lock(key)
        try:
            counter = distributed_map.get(key) + 1
            distributed_map.put(key, counter)
        finally:
            distributed_map.unlock(key)


# Функція для інкременту значення з використанням механізму заміни значень (replace_if_same)
def optimistic_replace(key):
    if (not distributed_map.contains_key(key)): distributed_map.put(key, 0)
    for _ in range(10000):
        while True:
            oldcounter = distributed_map.get(key)
            newcounter = oldcounter + 1
            if distributed_map.replace_if_same(key, oldcounter, newcounter): break


# Функція для інкременту значення з використанням атомарного лічильника Hazelcast
def atomic_counter(key):
    count = hz.cp_subsystem.get_atomic_long(key).blocking()
    for _ in range(10000):
        counter = count.add_and_get(1)
        distributed_map.put(key, counter)


if __name__ == "__main__":
    # Підключення до Hazelcast через Python-клієнта
    hz = hazelcast.HazelcastClient(cluster_name="massive-counter-cluster",
                                   client_name="lock_increment")
    distributed_map = hz.get_map("lab-distributed-map").blocking()
    print("Successful connection to Hazelcast!")
    
    # Використання ключа для розподіленого лічильника
    main_counter = 0
    print("Initial counter value:", main_counter)

    # Завдання до виконання
    tasks = [[with_lock, "1. Lock for Increment"],
             [no_lock, "2. No Lock for Increment"],
             [pessimistic_map, "3. Map Lock for Increment"],
             [optimistic_replace, "4. Replace if Same Increment"],
             [atomic_counter, "5. Atomic Counter Increment"]]

    for task, name in tasks:
        print(f"\n{'-' * 50}\nExecuting {name}...\n{'-' * 50}")
        
        start = time.time()
        threads = [threading.Thread(target=task, args=[task.__name__]) for _ in range(10)]
        
        for thread in threads: thread.start()
        for thread in threads: thread.join()
            
        print(f"Task '{name}' executed in: {time.time() - start:.2f} seconds")
        print(f"Final counter value: {distributed_map.get(task.__name__)}")
        print(f"{'-' * 50}\n")

    # Завершення роботи клієнта
    hz.shutdown()

Successful connection to Hazelcast!
Initial counter value: 0

--------------------------------------------------
Executing 1. Lock for Increment...
--------------------------------------------------
Task '1. Lock for Increment' executed in: 38.48 seconds
Final counter value: 100000
--------------------------------------------------


--------------------------------------------------
Executing 2. No Lock for Increment...
--------------------------------------------------
Task '2. No Lock for Increment' executed in: 62.10 seconds
Final counter value: 11003
--------------------------------------------------


--------------------------------------------------
Executing 3. Map Lock for Increment...
--------------------------------------------------
Task '3. Map Lock for Increment' executed in: 513.57 seconds
Final counter value: 100000
--------------------------------------------------


--------------------------------------------------
Executing 4. Replace if Same Increment...
---------